In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
import pandas as pd

# 크롬 옵션 설정 (헤드리스 모드로 설정하여 브라우저가 GUI 없이 실행되도록 함)
chrome_options = Options()
chrome_options.add_argument("--headless")  # GUI 없이 실행
chrome_options.add_argument("--no-sandbox")  # 샌드박스 모드 비활성화
chrome_options.add_argument("--disable-dev-shm-usage")  # /dev/shm 사용 비활성화

# 크롬 드라이버 설정 및 실행 (ChromeDriverManager를 사용하여 드라이버 자동 설치 및 관리)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.implicitly_wait(5)  # 암묵적 대기, 최대 5초까지 대기

try:
    # 카카오 맵 웹사이트 열기
    driver.get("https://map.kakao.com/")

    # 검색창 찾기 및 "스타벅스" 검색어 입력 후 검색
    search_box = driver.find_element(By.ID, "search.keyword.query")  # 검색창 요소 찾기
    search_box.send_keys("스타벅스")  # 검색창에 "스타벅스" 입력
    search_box.send_keys(Keys.RETURN)  # Enter 키를 눌러 검색 실행

    # 검색 결과가 로드될 때까지 1초 대기
    time.sleep(1)

    # 페이지 소스를 가져와서 BeautifulSoup으로 파싱
    page_source = driver.page_source  # 페이지 소스 가져오기
    soup = BeautifulSoup(page_source, 'html.parser')  # BeautifulSoup을 사용하여 HTML 파싱

    # 매장명과 링크를 저장할 리스트
    stores = []

    # 검색 결과에서 각 매장 정보 가져오기
    store_list = soup.find_all('li', class_='PlaceItem')  # 모든 매장 정보 요소 찾기

    for store in store_list:
        name_tag = store.find('a', class_='link_name')  # 매장명 요소 찾기
        link_tag = store.find('a', class_='moreview')  # 상세보기 링크 요소 찾기
        
        if name_tag and link_tag:  # 매장명과 링크가 모두 존재할 경우
            store_name = name_tag.text.strip()  # 매장명 텍스트 추출 및 공백 제거
            store_link = link_tag['href']  # 링크의 href 속성 추출
            stores.append({'매장명': store_name, '링크': store_link})  # 리스트에 딕셔너리 형태로 저장

    # 데이터프레임으로 변환
    df = pd.DataFrame(stores)  # pandas를 사용하여 리스트를 데이터프레임으로 변환

    # CSV 파일로 저장
    df.to_csv('starbucks_kakao.csv', index=False, encoding='utf-8-sig')  # 데이터프레임을 CSV 파일로 저장, UTF-8-sig 인코딩 사용

finally:
    # 드라이버 종료
    driver.quit()  # 브라우저 드라이버 종료


KeyboardInterrupt: 

In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# CSV 파일 읽기
df = pd.read_csv('./data/url.csv', encoding='utf-8')

# 셀레니움 웹드라이버 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 추가 정보 수집
additional_info = []

for index, row in df.iterrows():
    url = row['링크']
    driver.get(url)
    time.sleep(1)  # 페이지 로드 대기 시간 늘리기

    # 스크롤을 맨 아래로 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)  # 스크롤 후 대기
    
    # 다시 맨 위로 올리기
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(1)  # 스크롤 후 대기

    try:
        # 평점 크롤링
        try:
            rating = driver.find_element(By.CSS_SELECTOR, '#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > div.location_evaluation > a:nth-child(3) > span.color_b').text.strip()
        except:
            rating = 'N/A'

        # 평점 카운트 크롤링
        try:
            rating_count = driver.find_element(By.CSS_SELECTOR, '#mArticle > div.cont_evaluation > strong.total_evaluation > span').text.strip()
        except:
            rating_count = 'N/A'

        # 리뷰 개수 크롤링
        try:
            review_count = driver.find_element(By.CSS_SELECTOR, '#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > div.location_evaluation > a:nth-child(5) > span').text.strip()
        except:
            review_count = 'N/A'

        # 가능 여부 크롤링
        try:
            availability = driver.find_element(By.CSS_SELECTOR, '#mArticle > div.cont_essential > div.details_placeinfo > div:nth-child(7) > div').text.strip()
        except:
            availability = 'N/A'

        # 시설 정보 크롤링
        facilities_info = {}
        try:
            facilities_elements = driver.find_elements(By.CSS_SELECTOR, '#mArticle > div.cont_essential > div.details_placeinfo > div.placeinfo_default.placeinfo_facility > ul > li')
            for element in facilities_elements:
                try:
                    icon = element.find_element(By.TAG_NAME, 'span').get_attribute('class')
                    if 'ico_parking' in icon:
                        facilities_info['주차 공간'] = '가능'
                    elif 'ico_noparking' in icon:
                        facilities_info['주차 공간'] = '불가능'
                    elif 'ico_playroom' in icon:
                        facilities_info['놀이 공간'] = '가능'
                    elif 'ico_noplayroom' in icon:
                        facilities_info['놀이 공간'] = '불가능'
                    elif 'ico_handicapped' in icon:
                        facilities_info['휠체어 접근 가능'] = '가능'
                    elif 'ico_nohandicapped' in icon:
                        facilities_info['휠체어 접근 가능'] = '불가능'
                except:
                    pass
        except:
            pass

        # 영업시간 크롤링
        try:
            # 영업시간 버튼이 있는 경우
            try:
                driver.find_element(By.CSS_SELECTOR, '#mArticle > div.cont_essential > div.details_placeinfo > div:nth-child(3) > div > div.location_present > div > ul > li > a > span').click()
                time.sleep(2)  # 페이지 로드 대기

                # 첫 번째 경우: 두 개의 리스트 항목이 있는 경우
                try:
                    hours_elements = driver.find_elements(By.CSS_SELECTOR, '#mArticle > div.cont_essential > div.details_placeinfo > div:nth-child(3) > div > div.fold_floor > div > div > ul > li')
                    hours = ', '.join([hour.text.strip() for hour in hours_elements])
                except:
                    hours = 'N/A'

                # 두 번째 경우: 하나의 리스트 항목이 있는 경우
                try:
                    display_period_elements = driver.find_elements(By.CSS_SELECTOR, '#mArticle > div.cont_essential > div.details_placeinfo > div:nth-child(3) > div > div.fold_floor > div > div.displayPeriodList > ul > li')
                    display_period = ', '.join([period.text.strip() for period in display_period_elements])
                    hours = display_period if display_period else hours
                except:
                    pass
            except:
                # 영업시간 버튼이 없는 경우
                hours = driver.find_element(By.CSS_SELECTOR, '#mArticle > div.cont_essential > div.details_placeinfo > div:nth-child(3) > div > div > div > ul > li > span').text.strip()
        except:
            hours = 'N/A'

        # 평가 항목 크롤링 및 매핑
        evaluation_data = {
            '분위기': 'N/A',
            '맛': 'N/A',
            '친절': 'N/A',
            '가성비': 'N/A',
            '주차': 'N/A'
        }
        try:
            evaluation_elements = driver.find_elements(By.CSS_SELECTOR, '#mArticle > div.cont_evaluation > div.view_likepoint > span.chip_likepoint')
            for element in evaluation_elements:
                try:
                    label = element.find_element(By.CSS_SELECTOR, 'span.txt_likepoint').text.strip()
                    count = element.find_element(By.CSS_SELECTOR, 'span.num_likepoint').text.strip()
                    if label in evaluation_data:
                        evaluation_data[label] = count
                except:
                    pass
        except:
            pass

        # 시설 정보 필터링: 'N/A' 값은 제외
        facilities_info = {key: value for key, value in facilities_info.items() if value != 'N/A'}

        additional_info.append({
            '지점명': row['매장명'], 
            '평점': rating, 
            '평점 카운트': rating_count,
            '리뷰 개수': review_count,
            '가능 여부': availability, 
            '시설 정보': ', '.join([f"{key}: {value}" for key, value in facilities_info.items()]),
            '영업시간': hours,
            '분위기': evaluation_data['분위기'],
            '맛': evaluation_data['맛'],
            '친절': evaluation_data['친절'],
            '가성비': evaluation_data['가성비'],
            '주차': evaluation_data['주차']
        })
    
    except Exception as e:
        print(f"Error occurred at {url}: {e}")
        additional_info.append({
            '지점명': row['매장명'], 
            '평점': 'N/A', 
            '평점 카운트': 'N/A', 
            '리뷰 개수': 'N/A',
            '가능 여부': 'N/A', 
            '시설 정보': 'N/A', 
            '영업시간': 'N/A',
            '분위기': 'N/A',
            '맛': 'N/A',
            '친절': 'N/A',
            '가성비': 'N/A',
            '주차': 'N/A'
        })

# 브라우저 종료
driver.quit()

# 수집한 데이터를 DataFrame으로 변환 및 CSV 파일로 저장
df_additional = pd.DataFrame(additional_info)
df_additional.to_csv('./data/kakaomap_starbucks.csv', index=False, encoding='utf-8')

print('추가 정보가 data 폴더 안 kakaomap_starbucks.csv 파일에 저장되었습니다.')


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.127)
Stacktrace:
0   chromedriver                        0x0000000100652a80 chromedriver + 4385408
1   chromedriver                        0x000000010064b38c chromedriver + 4354956
2   chromedriver                        0x0000000100268b0c chromedriver + 281356
3   chromedriver                        0x0000000100243af0 chromedriver + 129776
4   chromedriver                        0x00000001002d0314 chromedriver + 705300
5   chromedriver                        0x00000001002e3438 chromedriver + 783416
6   chromedriver                        0x000000010029feec chromedriver + 507628
7   chromedriver                        0x00000001002a08c4 chromedriver + 510148
8   chromedriver                        0x000000010061a43c chromedriver + 4154428
9   chromedriver                        0x000000010061eea0 chromedriver + 4173472
10  chromedriver  